## Queue use Linked List

In [37]:
class Node(object):
    def __init__(self,value=None,next=None):
        self.value,self.next=value,next
        
class LinkedList(object):
    def __init__(self,maxsize=None):
        self.maxsize=maxsize
        self.root=Node()
        self.tailnode=None
        self.length=0
        
    def __len__(self):
        return self.length
    
    def append(self,value):
        if self.maxsize is not None and len(self)>=self.maxsize:
            raise Exception('LinkedList is Full')
        node=Node(value)
        tailnode=self.tailnode
        if tailnode is None:
            self.root.next=node
        else:
            tailnode.next=node
        self.tailnode=node
        self.length+=1
        
    def appendleft(self,value):
        if self.maxsize is not None and len(self)>=self.maxsize:
            raise Exception('LinkedList is Full')
        node=Node(value)
        headnode=self.root.next
        self.root.next=node
        node.next=headnode
        self.length+=1
        
    def iter_node(self):
        curnode=self.root.next
        tailnode=self.tailnode
        while curnode is not tailnode:
            yield curnode
            curnode=curnode.next
        yield curnode
        
    def __iter__(self):
        for node in self.iter_node():
            yield node.value
            
    def remove(self,value):
        prevnode=self.root
        curnode=self.root.next
        for curnode in self.iter_node():
            if curnode.value==value:
                prevnode.next=curnode.next
                del curnode
                self.length-=1
                return 1
            else:
                prevnode=curnode
        return -1
    
    def find(self,value):
        index=0
        for node in self.iter_node():
            if node.value==value:
                return index
            index+=1
        return -1
                
        
    def popleft(self):
        if self.root.next is None:
            raise Exception('pop from empty LinkedList')
        headnode=self.root.next
        self.root.next=headnode.next
        value=headnode.value
        self.length-=1
        del headnode
        return value
    
    def clear(self):
        for node in self.iter_node():
            del node
        self.root.next=None
        self.length=0
        
class EmptyError(Exception):
    pass

class Queue(object):
    def __init__(self,maxsize=None):
        self.maxsize=maxsize
        self._item_link_list=LinkedList()
        
    def __len__(self):
        return len(self._item_link_list)
    
    def push(self,value):
        return self._item_link_list.append(value)
    
    def pop(self):
        if len(self)<=0:
            raise EmptyError('Empty queue')
        return self._item_link_list.popleft()
    
def test_queue():
    q = Queue()
    q.push(0)
    q.push(1)
    q.push(2)

    assert len(q) == 3

    assert q.pop() == 0
    assert q.pop() == 1
    assert q.pop() == 2


In [38]:
test_queue()

## Queue use Array

In [19]:
class Array(object):
    def __init__(self,size=32):
        self._size=size
        self._items=[None]*size
        
    def __getitem__(self,index):
        return self._items[index]
    
    def __setitem__(self,index,value):
        self._items[index]=value
        
    def __len__(self):
        return self._size
    
    def clear(self,value=None):
        for i in range(len(self._items)):
            self._items[i]=value
            
    def __iter__(self):
        for i in self._items:
            yield i
            
class FullError(Exception):
    pass

class ArrayQueue(object):
    def __init__(self,maxsize=None):
        self.maxsize=maxsize
        self.head=0
        self.tail=0
        self.array=Array(maxsize)
        
    def push(self,value):
        if len(self)>=self.maxsize:
            raise FullError('queue full')
        self.array[self.head%self.maxsize]=value
        self.head+=1
        
    def pop(self):
        value=self.array[self.tail%self.maxsize]
        self.tail+=1
        return value
    
    def __len__(self):
        return self.head-self.tail
    
def test_queue():
    import pytest
    size=5
    q=ArrayQueue(size)
    for i in range(5):
        q.push(i)
        
    with pytest.raises(FullError) as excinfo:
        q.push(size)
    assert 'full' in str(excinfo.value)
    
    assert len(q)==5
    assert q.pop()==0
    assert q.pop()==1
    
    q.push(5)
    assert len(q)==4

In [20]:
test_queue()

## Deque use Double Linked List

In [46]:
class Node(object):

    def __init__(self, value=None, prev=None, next=None):
        self.value, self.prev, self.next = value, prev, next
    
        
class CircularDoubleLinkedList(object):
    def __init__(self, maxsize=None):
        self.maxsize = maxsize
        node = Node()
        node.next, node.prev = node, node
        self.root = node
        self.length = 0

    def __len__(self):
        return self.length

    def headnode(self):
        return self.root.next

    def tailnode(self):
        return self.root.prev

    def append(self, value):    
        if self.maxsize is not None and len(self) >= self.maxsize:
            raise Exception('LinkedList is Full')
        node = Node(value=value)
        tailnode = self.tailnode() or self.root

        tailnode.next = node
        node.prev = tailnode
        node.next = self.root
        self.root.prev = node
        self.length += 1

    def appendleft(self, value):
        if self.maxsize is not None and len(self) >= self.maxsize:
            raise Exception('LinkedList is Full')
        node = Node(value=value)
        if self.root.next is self.root:   # empty
            node.next = self.root
            node.prev = self.root
            self.root.next = node
            self.root.prev = node
        else:
            node.prev = self.root
            headnode = self.root.next
            node.next = headnode
            headnode.prev = node
            self.root.next = node
        self.length += 1

    def remove(self, node):     
        if node is self.root:
            return
        else:    #
            node.prev.next = node.next
            node.next.prev = node.prev
        self.length -= 1
        return node

    def iter_node(self):
        if self.root.next is self.root:
            return
        curnode = self.root.next
        while curnode.next is not self.root:
            yield curnode
            curnode = curnode.next
        yield curnode

    def __iter__(self):
        for node in self.iter_node():
            yield node.value

                    
        
class Deque(CircularDoubleLinkedList):   
    def pop(self):
        if len(self)==0:
            raise Exception('Empty')
        tailnode=self.tailnode()
        value=tailnode.value
        self.remove(tailnode)
        return value
    
    def popleft(self):
        if len(self)==0:
            raise Exception('Empty')
        headnode=self.headnode()
        value=headnode.value
        self.remove(headnode)
        return value
    
def test_deque():
    dq = Deque(5)
    dq.append(1)
    dq.append(2)
    assert list(dq) == [1, 2]

    dq.appendleft(0)
    assert list(dq) == [0, 1, 2]

    dq.pop()
    assert list(dq) == [0, 1]

    dq.popleft()
    assert list(dq) == [1]

    dq.pop()
    assert len(dq) == 0

In [47]:
test_deque()

## Stack use collection.deque

In [47]:
from collections import deque

class Stack(object):
    def __init__(self):
        self.d=deque()
        
    def push(self,value):
        return self.d.append(value)
        
    def pop(self):
        return self.d.pop()
        
def test_stack():
    s = Stack()
    s.push(0)
    s.push(1)
    s.push(2)

    assert s.pop() == 2
    assert s.pop() == 1
    assert s.pop() == 0

    assert s.pop()

In [ ]:
test_stack()

In [27]:
from collections import deque
d=deque()
d.append(1)
list(d)

[1]